# Bad Motor Movement Study

See [DESI-6135](https://desi.lbl.gov/DocDB/cgi-bin/private/ShowDocument?docid=6135)

In [1]:
%matplotlib inline

In [2]:
from pathlib import Path

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import pandas as pd

In [5]:
import fpoffline.const

In [6]:
import desietc.db

In [7]:
DESI = Path('/global/cfs/cdirs/desi')
if DESI.exists():
    DBCONFIG = DESI / 'engineering' / 'focalplane' / 'db.yaml'
    assert DBCONFIG.exists()
    DB = desietc.db.DB(config_name=DBCONFIG)
else:
    print(f'Not running at NERSC')

Not running at NERSC


## Exposures

Scan one of the `*_moves.csv` files from DESI-6135 to extract the list of expids used:

In [8]:
def get_expids(file=Path('p_only/expid_83267-(127-more)-83891_moves.csv')):
    f = pd.read_csv(file)
    expids = f.EXPOSURE_ID.unique()
    print(f'Found {len(expids)} exposures in the range {expids[0]} - {expids[-1]}')
    return expids

#expids = get_expids()

Fetch metadata for these exposures from the exposures DB:

In [9]:
def load_exps():
    sql = f'''
        select id,update_time,tileid,flavor,program
        from exposure.exposure where (id >= {expids[0]}) and (id <= {expids[-1]})
        order by id asc'''
    exps = DB.query(sql, maxrows=1000)
    sel = np.isin(exps.id, expids)
    return exps[sel]

#E = load_exps()

In [12]:
E.flavor.unique()

array(['science'], dtype=object)

In [13]:
E.program.unique()

array(['DARK', 'sv1elg', 'sv1lrgqso', 'sv1lrgqso2', 'BACKUP', 'BRIGHT',
       'sv1elgqso', 'sv1bgsmws'], dtype=object)

## Robots

Scan the p_only, t_only, t_and_p files from DESI-6135 to extract the list of robots analyzed:

In [14]:
def get_robots():
    robots = {}
    for folder in ('p_only', 't_only', 't_and_p'):
        file = list(Path(folder).glob('*.csv'))[0]
        f = pd.read_csv(file)
        posids = f.POS_ID.unique()
        print(f'Found {len(posids)} robots in {folder}')
        robots[folder] = posids
    return robots

R = get_robots()

Found 271 robots in p_only
Found 177 robots in t_only
Found 10 robots in t_and_p


In [15]:
assert set(R['p_only']).isdisjoint(set(R['t_only']))
assert set(R['p_only']).isdisjoint(set(R['t_and_p']))
assert set(R['t_only']).isdisjoint(set(R['t_and_p']))

In [16]:
posids = np.concatenate([r for r in R.values()])
rtypes = np.concatenate([np.full(len(r), k) for (k,r) in enumerate(R.values())])
print(f'Using {len(posids)} robots in total')

Using 458 robots in total


## Calibration

Load the most recent calibration record for each robot.  See [here](https://docs.google.com/spreadsheets/d/1e8yyjNFI9nCOT_KsJAxI3uzl8qSKqhuiDVXtXvxHNqM/edit#gid=836120262) for the calib schema.

In [18]:
def load_calib(name='badmotor-calib.csv'):
    
    if Path(name).exists():
        return pd.read_csv(name, index_col='pos_id', parse_dates=['time_recorded'])

    #before = pd.Timedelta(days_before, 'days')
    first = E.update_time.min()
    #last = E.update_time.min()    
    
    tables = []
    for petal_loc, petal_id in enumerate(fpoffline.const.PETAL_ID_MAP):
        table_name = f'posmovedb.positioner_calibration_p{petal_id}'
        sql = f'''
            select * from {table_name} where
            (time_recorded < timestamp '{first}')
        '''
        table = DB.query(sql, maxrows=100000)
        if len(table) > 0:
            print(f'Read {len(table)} rows for PETAL_LOC {petal_loc}')
        table['petal_loc'] = petal_loc
        table['location'] = 1000*petal_loc + table['device_loc']
        tables.append(table)
    calib = pd.concat(tables, axis='index', ignore_index=True)
    # Restrict to the posids used in DESI-6135.
    sel = np.isin(calib.pos_id, posids)
    calib = calib[sel]
    # Sort by increasing time.
    calib.sort_values('time_recorded', inplace=True, ignore_index=True)
    # Index by pos_id and only keep the most recent record for each robot.
    calib = calib.groupby('pos_id').last()
    # Save to csv
    calib.to_csv(name, index=True)
    
    print(f'Loaded {len(calib)} calibration records for {len(posids)} robots')
    return calib

C = load_calib()

In [60]:
C[(C.gear_calib_t < 1) & (C.gear_calib_p < 1)].index

Index(['M03455', 'M03583', 'M03653', 'M03676', 'M03684', 'M03733', 'M03816',
       'M03981', 'M03983', 'M04199'],
      dtype='object', name='pos_id')

Verify that all robots are classified functional and retracted:

In [52]:
assert np.all(~C.device_classified_nonfunctional & C.classified_as_retracted)

Check the classification of robots against their gear calibs:

In [10]:
def plot_gear_calibs():
    calib_t = np.asarray(C.gear_calib_t)
    calib_p = np.asarray(C.gear_calib_p)
    bins = np.linspace(-0.025, 1.025, 22)
    fig, axes = plt.subplots(2, 2, figsize=(10,6))
    for k, label in enumerate(R.keys()):
        c = f'C{k}'
        sel1 = rtypes == k
        sel2 = np.isin(C.index, posids[sel1])
        axes[0,0].scatter(calib_t[sel2], calib_p[sel2], s=10, c=c, label=label)
        axes[0,1].hist(calib_p[sel2], bins=bins, color=c, label=label)
        axes[1,0].hist(calib_t[sel2], bins=bins, color=c, label=label)

    axes[1,1].axis('off')
    axes[0,0].set(xlim=(bins[0],bins[-1]), ylim=(bins[0],bins[-1]), xlabel='gear_calib_t', ylabel='gear_calib_p')
    axes[0,1].set(xlim=(bins[0],bins[-1]), xlabel='gear_calib_p', yscale='log')
    axes[1,0].set(xlim=(bins[0],bins[-1]), xlabel='gear_calib_t', yscale='log')
    axes[1,0].legend()
    plt.tight_layout()
    plt.savefig('gear_calibs.png')

#plot_gear_calibs()

Check for any changes to the calibration while this data was being collected:

In [11]:
def check_calib_changes():

    first = E.update_time.min()
    last = E.update_time.max()
    
    tables = []
    for petal_loc, petal_id in enumerate(fpoffline.const.PETAL_ID_MAP):
        table_name = f'posmovedb.positioner_calibration_p{petal_id}'
        sql = f'''
            select * from {table_name} where
            (time_recorded >= timestamp '{first}') and
            (time_recorded <= timestamp '{last}')
        '''
        table = DB.query(sql, maxrows=1000)
        if len(table) > 0:
            print(f'Read {len(table)} rows for PETAL_LOC {petal_loc}')
        table['petal_loc'] = petal_loc
        table['location'] = 1000*petal_loc + table['device_loc']
        tables.append(table)
    calib = pd.concat(tables, axis='index', ignore_index=True)
    # Restrict to the posids used in DESI-6135.
    sel = np.isin(calib.pos_id, posids)
    calib = calib[sel]
    # Sort by increasing time.
    calib.sort_values('time_recorded', inplace=True, ignore_index=True)
    print(f'Found {len(calib)} new calib records')
    # Verify that none of the columns we care about have changed.
    cols = ['length_r1', 'length_r2', 'offset_x', 'offset_y', 'offset_t', 'offset_p', 'gear_calib_t', 'gear_calib_p',
            'physical_range_t', 'physical_range_p', 'device_classified_nonfunctional', 'classified_as_retracted']
    for i,row in calib.iterrows():
        old = C.loc[row.pos_id][cols]
        new = row[cols]
        assert np.all(old == new)
    print('No changes to columns we care about')

#check_calib_changes()

## Moves

Load all moves for these exposures:

In [13]:
def load_all_moves(name='badmotor-moves.csv'):

    if Path(name).exists():
        return pd.read_csv(name, parse_dates=['time_recorded'])

    tables = []
    for petal_loc, petal_id in enumerate(fpoffline.const.PETAL_ID_MAP):
        table_name = f'posmovedb.positioner_moves_p{petal_id}'
        move_cols = '''
            time_recorded,device_loc,pos_id,pos_t,pos_p,ctrl_enabled,move_cmd,move_val1,move_val2,log_note,
            exposure_id,exposure_iter,flags,ptl_x,ptl_y,obs_x,obs_y
            '''
        # We don't bother sorting by time_recorded since we do it globally after concatenating all petals.
        # We also don't restrict to the subset of expids used since it is faster to do this outside SQL.
        sql = f'''
            select {move_cols} from {table_name} where
                (exposure_id >= {expids[0]}) and (exposure_id <= {expids[-1]})
        '''
        table = DB.query(sql, maxrows=200000)
        if len(table) > 0:
            print(f'Read {len(table)} rows for PETAL_LOC {petal_loc}')
        table['location'] = 1000*petal_loc + table['device_loc']
        table.drop(columns='device_loc', inplace=True)
        tables.append(table)
    moves = pd.concat(tables, axis='index', ignore_index=True)
    # Restrict to the posids used in DESI-6135.
    sel = np.isin(moves.pos_id, posids)
    moves = moves[sel]
    # Sort by increasing time.
    moves.sort_values('time_recorded', inplace=True, ignore_index=True)
    # Save to csv.
    moves.to_csv(name, index=False)
    
    print(f'Loaded {len(moves)} moves for {len(expids)} exposures and {len(posids)} robots')
    return moves

%time all_moves = load_all_moves()

CPU times: user 389 ms, sys: 20.5 ms, total: 410 ms
Wall time: 410 ms


In [41]:
def process(moves=all_moves):
    
    # Verify that every row has angles.
    pos = ~(moves.pos_t.isna() | moves.pos_p.isna())
    assert np.all(pos)
    
    # Ignore rows where there was an FVC image but not

    # Verify that each row is either a move command or an FVC correction.
    cmd = ~moves.move_cmd.isna()
    spot = ~(moves.obs_x.isna() | moves.obs_y.isna())
    
    return moves[cmd & ~spot]
    
    #assert np.all((cmd & spot) | (~cmd & spot))

    return
    
    # Calculate and save the sum of commanded T,P moves.
    def sum_move(col_in, col_out):
        moves[col_out] = 0.
        valid = moves[col_in].notna()
        angle = moves[col_in].dropna().str.split('; | ').apply(lambda d: np.sum(np.array(list(map(float, d[1::2])))))
        moves.loc[valid, col_out] = angle
        moves.drop(columns=col_in, inplace=True) # Drop the original string column
    sum_move('move_val1', 'move_t')
    sum_move('move_val2', 'move_p')
    # Save to CSV
    moves.to_csv(name, index=False, compression='gzip')
    print(f'Wrote {name} with {len(moves)} rows.')
    
process()

,time_recorded,pos_id,pos_t,pos_p,ctrl_enabled,move_cmd,move_val1,move_val2,log_note,exposure_id,exposure_iter,flags,ptl_x,ptl_y,obs_x,obs_y,location
20459,2021-04-05 03:19:03.267140+00:00,M01722,-134.044443,134.995121,True,"QS=[69.37, 254.9049]",cruise -65.724; creep -0.000,creep -0.000; cruise 1.191,"req_posintTP=(-134.042, 134.995); req_ptlXYZ=(...",83414,0,NaN,NaN,NaN,NaN,NaN,4222
51364,2021-04-06 03:36:23.244370+00:00,M01778,9.924705,96.774622,True,"obsdXdY=[-2.705, 0.455]",creep 0.982; creep -0.000,creep -0.000; cruise -51.910,"req_posintTP=(9.925, 96.775); req_ptlXYZ=(175....",83525,1,NaN,NaN,NaN,NaN,NaN,9094
53576,2021-04-06 04:08:02.824995+00:00,M01758,-34.054714,93.054285,True,"QS=[240.126, 362.2476]",cruise 127.145; creep -0.000,creep -0.000; cruise -42.434,"req_posintTP=(-34.052, 93.054); req_ptlXYZ=(33...",83528,0,NaN,NaN,NaN,NaN,NaN,9454
65429,2021-04-07 02:56:36.207692+00:00,M01758,-34.878548,92.657905,True,"QS=[240.13, 362.2141]",cruise -33.718; creep -0.000,creep -0.000; cruise -41.661,"req_posintTP=(-34.876, 92.659); req_ptlXYZ=(33...",83713,0,NaN,NaN,NaN,NaN,NaN,9454
66799,2021-04-07 03:11:42.125605+00:00,M01778,74.722087,95.470537,True,"obsdXdY=[-1.623, -2.293]",creep 0.156; creep -0.000,creep -0.000; cruise -53.289,"req_posintTP=(74.722, 95.471); req_ptlXYZ=(176...",83714,1,NaN,NaN,NaN,NaN,NaN,9094
81398,2021-04-07 09:42:24.686572+00:00,M01722,-158.383787,116.249092,True,"obsdXdY=[3.151, -0.954]",cruise 6.769; creep -0.000,creep -0.000; cruise -61.512,"req_posintTP=(-158.386, 116.249); req_ptlXYZ=(...",83739,1,NaN,NaN,NaN,NaN,NaN,4222
95498,2021-04-08 05:15:50.022113+00:00,M01778,13.834590,86.995784,True,"QS=[228.555, 180.4452]",cruise -78.822; creep -0.000,creep -0.000; cruise -61.484,"req_posintTP=(13.834, 86.995); req_ptlXYZ=(176...",83865,0,NaN,NaN,NaN,NaN,NaN,9094
100495,2021-04-08 07:02:31.766563+00:00,M01722,-140.873391,128.133744,True,"obsdXdY=[2.684, 0.316]",cruise 7.415; creep -0.000,creep -0.000; cruise -49.702,"req_posintTP=(-140.872, 128.134); req_ptlXYZ=(...",83871,1,NaN,NaN,NaN,NaN,NaN,4222


In [35]:
all_moves.columns

Index(['time_recorded', 'pos_id', 'pos_t', 'pos_p', 'ctrl_enabled', 'move_cmd',
       'move_val1', 'move_val2', 'log_note', 'exposure_id', 'exposure_iter',
       'flags', 'ptl_x', 'ptl_y', 'obs_x', 'obs_y', 'location'],
      dtype='object')

In [39]:
def analyze(moves=M):
    # Calculate angles before each move.
    moves['prev_t'] = moves.pos_t - moves.move_t
    moves['prev_p'] = moves.pos_p - moves.move_p
    # Identify rows with successful FVC feedback.
    moves['fvc_feedback'] = (moves.ctrl_enabled &
                    moves.log_note.str.contains('handle_fvc_feedback') &
                    ~moves.log_note.str.contains('reject'))
    # Group moves by location.
    byloc = moves.groupby('location')
    # Calculate actual move angle for rows with fvc_update set, so current row is FVC update and previous row is big move.
    moves['moved_dt'] = moves.pos_t - byloc.prev_t.shift()
    moves['moved_dp'] = moves.pos_p - byloc.prev_p.shift()
    moves['req_dt'] = byloc.move_t.shift()
    moves['req_dp'] = byloc.move_p.shift()
    
analyze()

In [14]:
all_moves.columns

Index(['time_recorded', 'pos_id', 'pos_t', 'pos_p', 'ctrl_enabled', 'move_cmd',
       'move_val1', 'move_val2', 'log_note', 'exposure_id', 'exposure_iter',
       'flags', 'ptl_x', 'ptl_y', 'obs_x', 'obs_y', 'location'],
      dtype='object')

In [62]:
def plot(pos_id='M03455'):

    calib = C.loc[pos_id]
    moves = all_moves[all_moves.pos_id == pos_id]
    
    pos = ~moves.pos_t.isna()
    cmd = ~moves.move_cmd.isna()
    spot = ~moves.obs_x.isna()
    
    # It looks like the "handle_fvc_feedback" log note was not used in this older data?
    #feedback = moves.log_note.str.contains('handle_fvc_feedback')
    #print(np.count_nonzero(feedback))
    
    pd.set_option('display.max_rows', 500)
    return moves[['exposure_id','exposure_iter','ctrl_enabled','pos_t','move_cmd','obs_x','log_note']]
    
    assert np.all(pos)
    assert np.all(cmd | spot)

    no_fvc = ~moves.move_cmd.isna() & moves.obs_x.isna()
    fvc_only = moves.move_cmd.isna() & ~moves.obs_x.isna()
    normal = ~moves.move_cmd.isna() & ~moves.obs_x.isna()
    feedback = moves.log_note.str.contains('handle_fvc_feedback')
    
    print(len(moves), np.count_nonzero(no_fvc), np.count_nonzero(fvc_only), np.count_nonzero(normal), np.count_nonzero(feedback))

    return moves

plot()

,exposure_id,exposure_iter,ctrl_enabled,pos_t,move_cmd,obs_x,log_note
282,83267,0,True,175.315449,"QS=[165.608, 118.2488]",-115.026,"req_posintTP=(175.314, 85.433); req_ptlXYZ=(11..."
728,83267,1,True,174.290545,"obsdXdY=[0.473, -0.051]",-115.023,"req_posintTP=(174.293, 86.061); req_ptlXYZ=(11..."
1149,83268,0,True,175.338312,"QS=[165.608, 118.2479]",-115.017,"req_posintTP=(175.336, 85.429); req_ptlXYZ=(11..."
1623,83268,1,True,174.414598,"obsdXdY=[0.465, -0.061]",-115.017,"req_posintTP=(174.417, 85.908); req_ptlXYZ=(11..."
2034,83269,0,True,175.130416,"QS=[165.607, 118.2555]",-115.018,"req_posintTP=(175.130, 85.509); req_ptlXYZ=(11..."
2608,83269,1,True,174.004301,"obsdXdY=[0.454, -0.053]",-115.026,"req_posintTP=(174.006, 86.114); req_ptlXYZ=(11..."
3013,83270,0,True,175.151484,"QS=[165.607, 118.2546]",-115.019,"req_posintTP=(175.152, 85.505); req_ptlXYZ=(11..."
3404,83270,1,True,174.203630,"obsdXdY=[0.462, -0.051]",-115.017,"req_posintTP=(174.204, 86.056); req_ptlXYZ=(11..."
3939,83271,0,True,175.099102,"QS=[165.608, 118.2574]",-115.015,"req_posintTP=(175.100, 85.473); req_ptlXYZ=(11..."
4444,83271,1,True,174.100943,"obsdXdY=[0.452, -0.059]",-115.018,"req_posintTP=(174.102, 86.005); req_ptlXYZ=(11..."


In [26]:
all_moves.pos_id.unique()

array(['M01929', 'M05743', 'M01276', 'M01376', 'M06685', 'M01080',
       'M01831', 'M01102', 'M04619', 'M01861', 'M01482', 'M08230',
       'M06663', 'M01749', 'M03538', 'M04205', 'M03330', 'M03792',
       'M05074', 'M02323', 'M05089', 'M03060', 'M04319', 'M04052',
       'M03873', 'M03476', 'M03349', 'M01916', 'M04201', 'M03734',
       'M02767', 'M03915', 'M03443', 'M02889', 'M04197', 'M03784',
       'M05000', 'M05445', 'M03975', 'M03778', 'M04984', 'M04337',
       'M04318', 'M04383', 'M03513', 'M03963', 'M05398', 'M04262',
       'M03701', 'M03112', 'M04306', 'M03729', 'M04416', 'M04422',
       'M03572', 'M03675', 'M04356', 'M04031', 'M05090', 'M04999',
       'M04373', 'M03998', 'M04044', 'M05262', 'M02248', 'M03987',
       'M03703', 'M04199', 'M04084', 'M06460', 'M05952', 'M04606',
       'M06410', 'M03432', 'M02499', 'M02367', 'M06748', 'M03315',
       'M05684', 'M07263', 'M06812', 'M03151', 'M03037', 'M03122',
       'M03392', 'M07126', 'M01282', 'M02789', 'M06723', 'M072